<h3>1. Importing Necessary Libraries</h3>

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,StringType,IntegerType
from pyspark.sql.functions import*

<h3>2. Creating a Dataframe</h3>

In [2]:
spark = SparkSession.builder.appName('googleplaystoreanalysis').getOrCreate()
df=spark.read.load('googleplaystore.csv',format='csv',sep=',',header='true',escape='"',inferschema='true')
df.show(n=5, truncate=False)

+--------------------------------------------------+--------------+------+-------+----+-----------+----+-----+--------------+-------------------------+----------------+------------------+------------+
|App                                               |Category      |Rating|Reviews|Size|Installs   |Type|Price|Content Rating|Genres                   |Last Updated    |Current Ver       |Android Ver |
+--------------------------------------------------+--------------+------+-------+----+-----------+----+-----+--------------+-------------------------+----------------+------------------+------------+
|Photo Editor & Candy Camera & Grid & ScrapBook    |ART_AND_DESIGN|4.1   |159    |19M |10,000+    |Free|0    |Everyone      |Art & Design             |January 7, 2018 |1.0.0             |4.0.3 and up|
|Coloring book moana                               |ART_AND_DESIGN|3.9   |967    |14M |500,000+   |Free|0    |Everyone      |Art & Design;Pretend Play|January 15, 2018|2.0.0             |4.0.3 and

In [3]:
df.take(1)

[Row(App='Photo Editor & Candy Camera & Grid & ScrapBook', Category='ART_AND_DESIGN', Rating=4.1, Reviews='159', Size='19M', Installs='10,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='January 7, 2018', Current Ver='1.0.0', Android Ver='4.0.3 and up')]

<h4>2.1. Count number of rows in DataFrame</h4>

In [4]:
df.count()

10841

<h4>2.2. Printing Schema</h4>

In [5]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



<h3>3. Data Cleaning and PreProcessing</h3>

<h4>3.1. Dropping unecessary columns</h4>
<p>Removing the columns "Size", "Content Rating", "Last Updated", "Current Ver", and "Android Ver" from the DataFrame which are not required </p>

In [6]:
df=df.drop("Size","Content Rating","Last Updated","Current Ver","Android Ver")
df.show(5)

+--------------------+--------------+------+-------+-----------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|   Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+-----------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|    10,000+|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|   500,000+|Free|    0|Art & Design;Pret...|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 5,000,000+|Free|    0|        Art & Design|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644|50,000,000+|Free|    0|        Art & Design|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|   100,000+|Free|    0|Art & Design;Crea...|
+--------------------+--------------+------+-------+-----------+----+-----+--------------------+
only showing top 5 rows



In [7]:
df=df.na.drop(subset=['Rating'])
df=df.filter(df['Rating']<=5.0)

In [8]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Genres: string (nullable = true)



<h4>3.2. Normalization</h4>
<ul>
    <li>Converting "Reviews" column to integer type</li>
    <li>Removing non-numeric characters from the "Installs" column and converts it to integer type</li>
    <li>Removing the dollar sign from the "Price" column and converts it to integer type</li>
</ul>

In [9]:
from pyspark.sql.functions import regexp_replace,col
df=df.withColumn("Reviews",col("Reviews").cast(IntegerType()))\
.withColumn("Installs",regexp_replace(col("Installs"),"[^0-9]",""))\
.withColumn("Installs",col("Installs").cast(IntegerType()))\
            .withColumn("Price",regexp_replace(col("price"),"[$]",""))\
              .withColumn("Price",col("price").cast(IntegerType()))

In [10]:
df.show(5)

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|    0|Art & Design;Pret...|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 5000000|Free|    0|        Art & Design|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644|50000000|Free|    0|        Art & Design|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|  100000|Free|    0|Art & Design;Crea...|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
only showing top 5 rows



<h3>4. Creating View and using SQL for displaying the Dataframe</h3>

In [11]:
df.createOrReplaceTempView("apps")

In [12]:
spark.sql('''
SELECT * FROM apps
''').show(5)

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|    0|Art & Design;Pret...|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 5000000|Free|    0|        Art & Design|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644|50000000|Free|    0|        Art & Design|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|  100000|Free|    0|Art & Design;Crea...|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
only showing top 5 rows



<h3>5. Data Analysis</h3>

<h4>5.1. Top apps by Number of Reviews</h4>

In [13]:
spark.sql('''
SELECT App,SUM(Reviews) as TotalReviews
FROM apps
GROUP BY App
ORDER BY TotalReviews DESC
''').show(10)

+--------------------+------------+
|                 App|TotalReviews|
+--------------------+------------+
|           Instagram|   266241989|
|  WhatsApp Messenger|   207348304|
|      Clash of Clans|   179558781|
|Messenger – Text ...|   169932272|
|      Subway Surfers|   166331958|
|    Candy Crush Saga|   156993136|
|            Facebook|   156286514|
|         8 Ball Pool|    99386198|
|        Clash Royale|    92530298|
|            Snapchat|    68045010|
+--------------------+------------+
only showing top 10 rows



<h4>5.2. Top apps by installation and classifying whether paid or free</h4>

In [14]:
spark.sql('''
SELECT App,SUM(Installs) as TotalInstalls
FROM apps
GROUP BY App
ORDER BY TotalInstalls DESC
''').show(10)

+------------------+-------------+
|               App|TotalInstalls|
+------------------+-------------+
|    Subway Surfers|   6000000000|
|         Instagram|   4000000000|
|          Hangouts|   4000000000|
|      Google Drive|   4000000000|
|       Google News|   4000000000|
|     Google Photos|   4000000000|
|  Candy Crush Saga|   3500000000|
|             Gmail|   3000000000|
|WhatsApp Messenger|   3000000000|
|      Temple Run 2|   3000000000|
+------------------+-------------+
only showing top 10 rows



In [15]:
spark.sql('''
SELECT App,Type,SUM(Installs) as TotalInstalls
FROM apps
GROUP BY App,Type
ORDER BY TotalInstalls DESC
''').show(10)

+------------------+----+-------------+
|               App|Type|TotalInstalls|
+------------------+----+-------------+
|    Subway Surfers|Free|   6000000000|
|         Instagram|Free|   4000000000|
|      Google Drive|Free|   4000000000|
|          Hangouts|Free|   4000000000|
|     Google Photos|Free|   4000000000|
|       Google News|Free|   4000000000|
|  Candy Crush Saga|Free|   3500000000|
|WhatsApp Messenger|Free|   3000000000|
|             Gmail|Free|   3000000000|
|      Temple Run 2|Free|   3000000000|
+------------------+----+-------------+
only showing top 10 rows



<h4>5.3. Top Categories by Installations</h4>

In [16]:
spark.sql('''
SELECT Category,SUM(Installs) as TotalInstalls
FROM apps
GROUP BY Category
ORDER BY TotalInstalls DESC
''').show(10)

+------------------+-------------+
|          Category|TotalInstalls|
+------------------+-------------+
|              GAME|  35085862717|
|     COMMUNICATION|  32647241530|
|      PRODUCTIVITY|  14176070180|
|            SOCIAL|  14069841475|
|             TOOLS|  11450724500|
|            FAMILY|  10257701590|
|       PHOTOGRAPHY|  10088243130|
|NEWS_AND_MAGAZINES|   7496210650|
|  TRAVEL_AND_LOCAL|   6868859300|
|     VIDEO_PLAYERS|   6221897200|
+------------------+-------------+
only showing top 10 rows



<h4>5.4. Top apps with Highest Prices</h4>

In [17]:
spark.sql('''
SELECT App,SUM(Price) as TotalCost
FROM apps
WHERE Type='Paid'
GROUP BY App
ORDER BY TotalCost DESC
''').show(10)

+--------------------+---------+
|                 App|TotalCost|
+--------------------+---------+
|I'm Rich - Trump ...|      400|
|      I am Rich Plus|      399|
|  I AM RICH PRO PLUS|      399|
|   I Am Rich Premium|      399|
|most expensive ap...|      399|
|          I am Rich!|      399|
|       I Am Rich Pro|      399|
|  I am rich(premium)|      399|
|           I am Rich|      399|
|         💎 I'm rich|      399|
+--------------------+---------+
only showing top 10 rows



<h4>5.5 Highest rated apps</h4>

In [18]:
spark.sql('''
SELECT App,Rating
FROM apps
ORDER BY Rating DESC
''').show(10000)

+----------------------------------+------+
|                               App|Rating|
+----------------------------------+------+
|              Hojiboy Tojiboyev...|   5.0|
|              American Girls Mo...|   5.0|
|                      Awake Dating|   5.0|
|              Spine- The dating...|   5.0|
|              Girls Live Talk -...|   5.0|
|              Online Girls Chat...|   5.0|
|              Speeding Joyride ...|   5.0|
|                  SUMMER SONIC app|   5.0|
|                        Prosperity|   5.0|
|              Mindvalley U Tall...|   5.0|
|                      Eternal life|   5.0|
|              Super Hearing Sec...|   5.0|
|                    FHR 5-Tier 2.0|   5.0|
|                      Sway Medical|   5.0|
|                    Labs on Demand|   5.0|
|              Dermatology Atlas...|   5.0|
|                   Tablet Reminder|   5.0|
|                  Galaxies of Hope|   5.0|
|               KBA-EZ Health Guide|   5.0|
|                      Foothills

<h4>5.6. Number of apps in each Genre</h4>

In [19]:
spark.sql('''
SELECT Genres, COUNT(*) as TotalApps
FROM apps
GROUP BY Genres
ORDER BY TotalApps DESC
''').show(10)

+-------------+---------+
|       Genres|TotalApps|
+-------------+---------+
|        Tools|      733|
|Entertainment|      533|
|    Education|      468|
|       Action|      358|
| Productivity|      351|
|      Medical|      350|
|       Sports|      333|
|Communication|      328|
|      Finance|      323|
|  Photography|      317|
+-------------+---------+
only showing top 10 rows



<h4>5.7. Top Categories by Average Rating</h4>

In [20]:
spark.sql('''
SELECT Category, AVG(Rating) as AvgRating
FROM apps
GROUP BY Category
ORDER BY AvgRating DESC
''').show()

+-------------------+------------------+
|           Category|         AvgRating|
+-------------------+------------------+
|             EVENTS| 4.435555555555557|
|          EDUCATION| 4.389032258064517|
|     ART_AND_DESIGN| 4.358064516129031|
|BOOKS_AND_REFERENCE| 4.346067415730338|
|    PERSONALIZATION| 4.335987261146501|
|          PARENTING| 4.300000000000001|
|               GAME|4.2863263445761195|
|             BEAUTY| 4.278571428571428|
| HEALTH_AND_FITNESS|4.2771043771043775|
|           SHOPPING| 4.259663865546221|
|             SOCIAL| 4.255598455598457|
|            WEATHER| 4.243999999999999|
|             SPORTS| 4.223510971786835|
|       PRODUCTIVITY| 4.211396011396012|
|     HOUSE_AND_HOME| 4.197368421052633|
|             FAMILY| 4.192272467086437|
|        PHOTOGRAPHY| 4.192113564668767|
|  AUTO_AND_VEHICLES|  4.19041095890411|
|            MEDICAL|  4.18914285714286|
| LIBRARIES_AND_DEMO| 4.178461538461538|
+-------------------+------------------+
only showing top

<h4>5.8. Price Distribution of Paid Apps</h4>

In [21]:
spark.sql('''
SELECT Price, COUNT(*) as count
FROM apps
WHERE Type='Paid'
GROUP BY Price
ORDER BY CAST(Price AS FLOAT) DESC
''').show(10)

+-----+-----+
|Price|count|
+-----+-----+
|  400|    1|
|  399|   11|
|  389|    1|
|  379|    1|
|  299|    1|
|   79|    2|
|   39|    1|
|   37|    1|
|   33|    2|
|   29|    6|
+-----+-----+
only showing top 10 rows



<h4>5.9. Average Rating By Price Range</h4>

In [22]:
spark.sql('''
    SELECT 
        CASE 
            WHEN Price = '0' THEN 'Free' 
            WHEN Price BETWEEN '1' AND '50' THEN '1-50$'
            WHEN Price BETWEEN '51' AND '100' THEN '51-100$'
            WHEN Price BETWEEN '101' AND '200' THEN '101-200$'
            WHEN Price BETWEEN '201' AND '300' THEN '201-300$'
            WHEN Price BETWEEN '301' AND '400' THEN '301-400$'
            WHEN Price >= 401 THEN 'Above 400$'
        END AS PriceRange,
        AVG(Rating) AS AvgRating
    FROM apps
    GROUP BY PriceRange
    ORDER BY PriceRange
''').show()

+----------+-----------------+
|PriceRange|        AvgRating|
+----------+-----------------+
|     1-50$| 4.27036328871893|
|  201-300$|              3.8|
|  301-400$|3.871428571428571|
|   51-100$|              4.6|
|      Free|4.187559483344679|
+----------+-----------------+



<h4>5.10. Average Rating of Paid vs Free Apps</h4>

In [23]:
spark.sql('''
SELECT Type, AVG(Rating) as AvgRating
FROM apps
GROUP BY Type
''').show()

+----+-----------------+
|Type|        AvgRating|
+----+-----------------+
|Free|4.186202546163562|
|Paid|4.266615146831529|
+----+-----------------+

